In [20]:
#from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

from twodlearn.tf_lib.Feedforward import LinearLayer

import sys
working_dir= '/home/marinodl/dl_projects/nlp/sentiment_analysis/'
sys.path.insert(0, working_dir)


import math

#### Configuration Variables

In [21]:
Allow_Valid= True
Valid_Percentage= 0.1 # i.e. 10% of the data will be used for validation

pos_net_name= '_pos'
neg_net_name= '_neg'

activation_function='tanh'
num_nodes = [500,500,500] #64 500, num_nodes: Nodes for the LSTM cell
alpha = 0.01 #10.1 #0.1
beta = 0.001 #10.1 #0.01
lambda_w = 0.00001

dropout_cons = 1.0

Allow_Bias= False 

learning_rate= 0.0001      # 0.001
grad_clip_thresh= 0.1       # 0.00001

current_run= 1
batch_size= 64 #64
num_unrollings= 100 #100

batch_size_val= 64 #len(valid_text_pos)/num_unrollings_val #500

comment='_01alpha_noDropout_LcLpLcp_100unrol'

In [22]:
model_version = 'L'+str(len(num_nodes))
print("num_nodes:",num_nodes)
print("alpha:",alpha, ", beta:",beta,", lambda_w:", lambda_w)
print("num_unrollings:",num_unrollings, ", batch_size:",batch_size,", batch_size_val:", batch_size_val)
print("learning_rate:", learning_rate, 'grad_clip_thresh:', grad_clip_thresh)

num_nodes: [500, 500, 500]
alpha: 0.01 , beta: 0.001 , lambda_w: 1e-05
num_unrollings: 100 , batch_size: 64 , batch_size_val: 64


# 1. Load training dataset

In [23]:
#with open('/home/marinodl/datasets/nlp/without_html/positive.txt', 'r') as file_handler:
with open('/home/marinodl/datasets/nlp/imdb/train/imdb_positive.txt', 'r', errors='ignore') as file_handler:
    text_pos = file_handler.read()
    
#with open('/home/marinodl/datasets/nlp/without_html/negative.txt', 'r') as file_handler:
with open('/home/marinodl/datasets/nlp/imdb/train/imdb_negative.txt', 'r', errors='ignore') as file_handler:
    text_neg = file_handler.read()

len_text= min(len(text_pos), len(text_neg))//2 #CAREFUL

print(len_text) 

7985367


#### Creation of training, validation and testing sets

In [24]:
valid_size_pos = int(math.floor(len_text)*0.1)
valid_text_pos = text_pos[:valid_size_pos]
train_text_pos = text_pos[valid_size_pos:len_text] # excluding validation set
#train_text_pos = text_pos[:len_text]               # including validation set
train_size_pos = len(train_text_pos)

print(valid_size_pos)
print(train_size_pos)

798536
7186831


In [25]:
valid_size_neg = int(math.floor(len_text*0.1))
valid_text_neg = text_neg[:valid_size_neg]
train_text_neg = text_neg[valid_size_neg:len_text] # excluding validation set
#train_text_neg = text_neg[:len_text]               # including validation set
train_size_neg = len(train_text_neg)               
print(train_size_neg)

7186831


# 2. Batch generators

In [26]:
from vocabulary_coding_simple import *

vc= Vocabulary('')
num_inputs=  vc.vocabulary_size
num_outputs= vc.vocabulary_size

In [27]:
print([vc.char2id(i) for i in 'abc z.[]'])
print([vc.id2char(i) for i in [1, 2, 3, 0, 26, 27, 0, 0]])

[1, 2, 3, 0, 26, 0, 0, 0]
['a', 'b', 'c', ' ', 'z', ' ', ' ', ' ']


In [28]:
class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
    
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vc.vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, vc.char2id(self._text[self._cursor[b]])] = 1.0    # the letter pointed by the cursor is converted to 1-hot encoding
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size  # Here, the cursor is increased
        return batch
    
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch] # include last batch from previous array
        #batches = list()
        for step in range(self._num_unrollings):
            # each call of _next_batch() increases the cursor by one
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

In [29]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10 # this is to prevent that log() returns minus infinity
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vc.vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vc.vocabulary_size])
    return b/np.sum(b, 1)[:,None]

Get supervised-batch from positive and negative texts

In [30]:
batches_pos = BatchGenerator(train_text_pos, batch_size//2, num_unrollings)
batches_neg = BatchGenerator(train_text_neg, batch_size//2, num_unrollings)

def GetNextBatch_sentiment(rand_shuffle= False):
    ''' Generates a batch of sentences and its corresponding labels for classification
        Returns:
            X: list of positive and negative sentences, the format is the same that the batches generated from the 
               positive and negative texts. len(X)= num_unrrolings, X[i].shape: [batch_size x vocabulary_size]
            y: class to which each sentence belong. The format is a vector of size [batch_size x 1]
    '''
    batch_pos = batches_pos.next()
    batch_neg = batches_neg.next()
        
    y_aux= np.zeros(shape=[batch_size,1], dtype=np.float) # y_aux= class for each one of the sentences
    y_aux[0:int(batch_size//2),0]= 1 # positive samples are denoted by 1, negative by 0
    
    # TODO: ADD MULTIPLE CLASSES
    if rand_shuffle:
        rand_ind= np.random.permutation(batch_size) # rand_ind= random shuffling for the sentences
        y_aux=y_aux[rand_ind,:]
    
    X= list()
    #y= list()
    # TODO: try to do not penalize mistakes being done in the first samples of the unrolling
    # start_ind= math.floor(0.5*len(batch_pos))
    for x_ind in range(len(batch_pos)):  # the length of batch_pos is the number of unrollings
        # Concatenate positive and negative batches and arrange them according to rand_ind
        if rand_shuffle:
            X.append( np.concatenate((batch_pos[x_ind],batch_neg[x_ind]),0)[rand_ind,:] )
        else:
            X.append( np.concatenate((batch_pos[x_ind],batch_neg[x_ind]),0) )
        
    return X, y_aux

batch_X, batch_y = GetNextBatch_sentiment()

### For Validation

In [31]:
num_unrollings_val= 100 #num_unrollings #10 #num_unrollings 
print('batch_size_val:', batch_size_val, ' suggested:', len(valid_text_pos)/num_unrollings_val)

batches_pos_val = BatchGenerator(valid_text_pos, batch_size_val//2, num_unrollings_val)
batches_neg_val = BatchGenerator(valid_text_neg, batch_size_val//2, num_unrollings_val)
#batches_pos_val = BatchGenerator(train_text_pos, batch_size_val/2, num_unrollings_val)
#batches_neg_val = BatchGenerator(train_text_neg, batch_size_val/2, num_unrollings_val)


def Get_Val():
    batch_pos = batches_pos_val.next()
    batch_neg = batches_neg_val.next()
        
    y_aux= np.zeros(shape=[batch_size_val,1], dtype=np.float) # y_aux= class for each one of the sentences
    y_aux[0:int(batch_size_val//2),0]= 1
    
    # TODO: ADD MULTIPLE CLASSES
    rand_ind= np.random.permutation(batch_size_val) # rand_ind= random shuffling for the sentences
    
    y_aux=y_aux[rand_ind,:]
    
    X= list()
    #y= list()
    # TODO: try to do not penalize mistakes being done in the first samples of the unrolling
    # start_ind= math.floor(0.5*len(batch_pos))
    for x_ind in range(len(batch_pos)):  # the length of batch_pos is the number of unrrolings
        # Concatenate positive and negative batches and arrange them according to rand_ind
        X.append( np.concatenate((batch_pos[x_ind],batch_neg[x_ind]),0)[rand_ind,:] )
        
    return X, y_aux

valid_X, valid_y = Get_Val()

batch_size_val: 64  suggested: 7985.36


# 3. Model for positive and negative reviews

In [32]:
from lstm_model_V2 import *

class myLstmNet(LstmNet):
    def get_extra_inputs(self, i, h_list, state_list):
        #print('OK:', len(h_list))
        return i
    
    def evaluate_final_output(self, outputs_list, inputs_list, h_list ):
        ''' Calculates the final output of the neural network, usually it is just a linear transformation
        
        outputs_list: list with the outputs from the last lstm cell
        inputs_list: list of inputs to the network
        h_list: list with all hidden outputs from all the cells
        '''
        ''''''
        all_hidden = list()
        #print('n_unrollings:', len(h_list)) # DELETE !!!
        #print('n_layers:', len(h_list[0])) # DELETE !!!
        
        for t in h_list: # go trough each time step
            all_hidden.append( tf.concat(1,t) )
        return self.out_layer.evaluate(tf.concat(0, all_hidden))  
        
        
        # Original:
        #return self.out_layer.evaluate(tf.concat(0, outputs_list))
    
if len(num_nodes)>1:
    n_extra= [num_inputs for i in range(len(num_nodes)+1)]
    n_extra[0]= 0
    n_extra[-1]= sum(num_nodes) - num_nodes[-1]
else:
    n_extra= [0,0]

In [33]:
class ModelSetup:
    
    def __init__( self, pos_net, neg_net, w, b, batch_size, num_unrollings, drop_prob_list, name=''):
    
        # 1. Create placeholders for inputs 
        self.X = list()
        for iaux in range(num_unrollings + 1):
            self.X.append(tf.placeholder(tf.float32, shape=[batch_size, vc.vocabulary_size], 
                                             name= name+'X_i'+str(iaux)+'_All'))
        aux_inputs = self.X[:num_unrollings]
        aux_labels = self.X[1:]  # inputs shifted by one time step.

        # Create a list for store the placeholders for the labels
        self.labels = tf.placeholder(tf.float32, shape=[batch_size, 1])
       

        # -------------------- unrolling of the network --------------------------- # 
        self.pos_unroll, _= pos_net.unrolling_setup( batch_size, num_unrollings, 
                                                     inputs_list= aux_inputs,
                                                     labels_list= aux_labels,
                                                     drop_prob_list= drop_prob_list,
                                                     reset_between_unrollings= True,
                                                   )

        self.neg_unroll, _= neg_net.unrolling_setup( batch_size, num_unrollings, 
                                                     inputs_list= aux_inputs,
                                                     labels_list= aux_labels,
                                                     drop_prob_list= drop_prob_list,
                                                     reset_between_unrollings= True,
                                                   ) 


        # Classifier.
        # error_per_sample is a vector, its shape is changed to have each unrolling in separate columns
        output_pos= tf.reshape(self.pos_unroll.error_per_sample,[num_unrollings,batch_size])  
        output_neg= tf.reshape(self.neg_unroll.error_per_sample,[num_unrollings,batch_size]) 

        output_pos_mean= tf.reduce_mean(output_pos, reduction_indices= 0) 
        output_neg_mean= tf.reduce_mean(output_neg, reduction_indices= 0) 

        output_mean= tf.transpose(tf.pack( [ output_pos_mean, output_neg_mean ]))


        self.logits = tf.nn.xw_plus_b( output_mean , w, b )


        self.error_per_sample= tf.nn.sigmoid_cross_entropy_with_logits( self.logits, self.labels )
        
        # prediction error
        #Lp = tf.reduce_mean( tf.mul(self.labels, output_pos_mean) + tf.mul(self.labels-1, output_neg_mean))
        Lp = tf.reduce_mean( tf.mul(tf.squeeze(self.labels), output_pos_mean) + 
                             tf.mul(tf.squeeze(1-self.labels), output_neg_mean)
                           )
        # counter-prediction penalty
        Lcp = tf.reduce_mean( tf.mul(tf.squeeze(1-self.labels), tf.exp(-output_pos_mean)) + 
                              tf.mul(tf.squeeze(self.labels), tf.exp(-output_neg_mean))
                            )
        # regularization
        l2_c= tf.nn.l2_loss(w)
        
        #self.loss = tf.reduce_mean( self.error_per_sample ) + alpha*Lp + beta*(1.0/Lcp)
        #self.loss = tf.reduce_mean( self.error_per_sample ) + alpha*Lp - beta*(Lcp)
        self.loss = tf.reduce_mean( self.error_per_sample ) + alpha*Lp + beta*Lcp + lambda_w*l2_c

In [34]:

graph = tf.Graph()
with graph.as_default():
    # For dropout
    drop_prob = tf.placeholder(tf.float32)
    drop_prob_list = [ drop_prob for i in range(len(num_nodes)+1)]
    drop_prob_list[0]= None
    
     
    # 1. Define positive and negative neural networks
    pos_net= myLstmNet( num_inputs, num_nodes, num_outputs, n_extra= n_extra,
                        afunction=activation_function, 
                        LstmCell= AlexLstmCell,
                        name= pos_net_name)
        
    neg_net= myLstmNet( num_inputs, num_nodes, num_outputs, n_extra= n_extra,
                        afunction=activation_function, 
                        LstmCell= AlexLstmCell,
                        name= neg_net_name)
    
    # Classifier weights and biases.
    #w = tf.Variable(tf.truncated_normal([2, 1], -0.1, 0.1), name=('w_class')) # unconstrained w
    #w = tf.Variable(tf.constant( [[-2.0], [2.0]]), name=('w_class'), trainable=False) # fixed w
    
    #w = tf.Variable(tf.truncated_normal([1, 1], 0.3, 0.4), name=('w_class')) # unconstrained w
    w = tf.Variable(tf.constant( [[0.3]] ), name=('w_class')) # fixed w
    w = tf.concat(0, [-w, w])
    
    b = tf.Variable(tf.zeros([1]), name=('b_class'), trainable=Allow_Bias)
    
    
    
    # 2. Define unrolling for training
    train= ModelSetup( pos_net, neg_net, w, b, batch_size, num_unrollings, 
                       drop_prob_list, 
                       name='train_')
       
    # 3. Define unrolling for validation
    valid= ModelSetup( pos_net, neg_net, w, b, batch_size_val, num_unrollings_val, 
                       drop_prob_list= [None, None, None, None, None], 
                       name='train_')
   
    # 4. Define unrolling for testing
    pos_gen_test, _= pos_net.unrolling_setup( 1, 1, drop_prob_list= [None, None, None, None, None] )
    neg_gen_test, _= neg_net.unrolling_setup( 1, 1, drop_prob_list= [None, None, None, None, None] )
    
          
    
    # 5. Define optimizer    
    # 1. specify the optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate) # ADAM 0.001
    
    # 2. get the gradients and variables
    # grads_and_vars is a list of tuples (gradient, variable). 
    grads_and_vars = optimizer.compute_gradients(train.loss) 
    gradients, v = zip(*grads_and_vars)
    
    # 3. process the gradients
    gradients, _ = tf.clip_by_global_norm(gradients, grad_clip_thresh)  #1.25 #0.025 #0.001(last used)
    # 4. apply the gradients to the optimization procedure
    optimizer = optimizer.apply_gradients( zip(gradients, v) ) # ADAM
    
    
    # Saver
    saver = tf.train.Saver()

# 4. Train Energy predictor model

In [35]:
reload_pos= False
load_pos_file=  working_dir+"weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                "1r_pos.ckpt"
    
reload_neg= False
load_neg_file= working_dir+"weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                "1r_neg.ckpt"

reload_all= True
load_all_file=  working_dir+"weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                str(current_run-1) +"r_All"+comment+".ckpt";

save_all= True
save_all_file=  working_dir+"weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                str(current_run) +"r_All"+comment+".ckpt";

In [ ]:
if current_run==1:
    reload_all= False;
else:
    reload_pos= False;
    reload_neg= False;

In [ ]:
num_steps = 13000 
summary_frequency = 50
n_valid_tests = 100
n_characters_step= num_unrollings*batch_size

aux_print=0
with tf.Session(graph=graph) as session:
    # -------------------------------- Load weigths from file, or initialize variables ------------------------------------
    if reload_all:
        saver.restore(session, load_all_file)
        #session.run( global_step.assign(0) ) # SGD
        
    elif (reload_pos and reload_neg):
        tf.initialize_all_variables().run()
        pos_net.saver.restore(session, load_pos_file)
        neg_net.saver.restore(session, load_neg_file)
        print('Weights for positive and negative networks loaded')
    else:
        tf.initialize_all_variables().run()
        print('Weights Initialized')
    
    # ------------------------------------------- Training loop --------------------------------------------------
    print('step | Train_err, Train_mis | Valid_err, Valid_mis | loss')
    
    mean_loss = 0
    for step in range(num_steps):
        # 1. Get next batch
        batch_X, batch_y = GetNextBatch_sentiment()        
        
        # 2. Setup feed dictionary for network 1 and network 2
        feed_dict = dict()
        # For dropout
        feed_dict[drop_prob] = dropout_cons
        # Inroduce labels
        feed_dict[train.labels] = batch_y
        # Introduce inputs
        for i in range(num_unrollings+1):
            feed_dict[train.X[i]] = batch_X[i]
                               
        # 3 Run optimizer
        #_, l, lr = session.run( [optimizer, loss_train, learning_rate], feed_dict=feed_dict) # SGD
        _, l = session.run( [optimizer, train.loss], feed_dict=feed_dict) # ADAM
        
        mean_loss += l
        
        # --------------------------------------------- logging ------------------------------------------------
        if train_size_pos<aux_print :
            print('+'*80)
            print('')
            aux_print = 0
        else:
            aux_print += n_characters_step
        
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
                                
            # ---------- Print loss in validation dataset ---------
            if Allow_Valid:
                '''train mis classification'''        
                prediction_train = session.run( [tf.nn.sigmoid(train.logits)], feed_dict=feed_dict)          
                mis_pred_train = np.sum( np.not_equal( np.greater(prediction_train, 0.5), np.greater(batch_y, 0.5)) )
                
                
                '''valid mis classification'''
                mis_pred_val = 0
                for step_valid in range(n_valid_tests):
                    # 1. Get next batch
                    batch_X_val, batch_y_val = Get_Val()        
                    feed_dict_val = dict()
                    # For dropout
                    feed_dict_val[drop_prob] = 1.0
                    # Inroduce labels
                    feed_dict_val[valid.labels] = batch_y_val
                    # Introduce inputs
                    for i in range(num_unrollings_val+1):
                        feed_dict_val[valid.X[i]] = batch_X_val[i]
                    # 2. Get prediction
                    prediction_val = session.run( [tf.nn.sigmoid(valid.logits)], feed_dict=feed_dict_val)          
                    mis_pred_val = mis_pred_val*step_valid
                    mis_pred_val += np.sum( np.not_equal( np.greater(prediction_val, 0.5), np.greater(batch_y_val, 0.5)) )
                    mis_pred_val = mis_pred_val/(step_valid+1)
                    #
                    if (mis_pred_val>0.3) and (step_valid==0):
                        break                        
                    elif (step_valid==0):
                        print('Low validation error reached:', mis_pred_val)
                
                ''' weights and bias for classficator '''
                # print values for b and w
                b_t= b.eval()
                w_t= w.eval()
                
                print('%d | %f, %d | %f, %d | %f | w:[%f,%f], b:%f' % (step, 
                                                     mis_pred_train/batch_y.shape[0], mis_pred_train,
                                                     mis_pred_val/batch_y_val.shape[0], mis_pred_val,
                                                     mean_loss,
                                                     w_t[0], w_t[1], b_t[0]
                                                    ))                
                #if(mis_pred_val/batch_y_val.shape[0] < 0.20):
                #    break
                
            mean_loss = 0
            
    if save_all:
        save_path = saver.save(session, save_all_file)
        print("LSTM cell Model for negative reviews saved in file: %s" % save_path)
        

Weights Initialized
step | Train_err, Train_mis | Valid_err, Valid_mis | loss
0 | 0.515625, 33 | 0.562500, 36 | 0.731230 | w:[-0.300100,0.300100], b:0.000000
50 | 0.453125, 29 | 0.453125, 29 | 0.728945 | w:[-0.298915,0.298915], b:0.000000
100 | 0.453125, 29 | 0.390625, 25 | 0.724589 | w:[-0.298561,0.298561], b:0.000000
150 | 0.453125, 29 | 0.437500, 28 | 0.722337 | w:[-0.300063,0.300063], b:0.000000
200 | 0.546875, 35 | 0.468750, 30 | 0.721327 | w:[-0.302206,0.302206], b:0.000000
250 | 0.515625, 33 | 0.406250, 26 | 0.721222 | w:[-0.304880,0.304880], b:0.000000
300 | 0.453125, 29 | 0.546875, 35 | 0.720682 | w:[-0.307684,0.307684], b:0.000000
350 | 0.390625, 25 | 0.484375, 31 | 0.720408 | w:[-0.311387,0.311387], b:0.000000
400 | 0.390625, 25 | 0.578125, 37 | 0.720855 | w:[-0.314017,0.314017], b:0.000000
450 | 0.406250, 26 | 0.468750, 30 | 0.720155 | w:[-0.317306,0.317306], b:0.000000


## Generate some samples from positive and negative networks

In [ ]:
with tf.Session(graph=graph) as session:
    # Load weigths from file, or initialize variables
    tf.initialize_all_variables().run()
    saver.restore(session, save_all_file)
    print('Weights loaded')
        
    # print values for b and w
    b_t= b.eval()
    w_t= w.eval()
    #b_t= session.run(b)
    print('b=', b_t)
    print('w=', w_t)
    
    
    # For positive network
    print('-'*30 + 'pos' + '-'*30)
    for _ in range(5):
        feed = sample(random_distribution())
        sentence = vc.prob2char(feed)[0]
        pos_gen_test.reset_saved_out_state.run()
        for _ in range(70):
            prediction = tf.nn.softmax(pos_gen_test.y).eval({pos_gen_test.inputs_list[0]: feed}) # feed is a 1-hot encoding
            feed = sample(prediction)  # sample returns a 1-hot encoding
            sentence += vc.prob2char(feed)[0]
        print(sentence)
    print('-' * 80)
    
    # For negative network
    print('-'*30 + 'neg' + '-'*30)
    for _ in range(5):
        feed = sample(random_distribution())
        sentence = vc.prob2char(feed)[0]
        neg_gen_test.reset_saved_out_state.run()
        for _ in range(70):
            prediction = tf.nn.softmax(neg_gen_test.y).eval({neg_gen_test.inputs_list[0]: feed}) # feed is a 1-hot encoding
            feed = sample(prediction)  # sample returns a 1-hot encoding
            sentence += vc.prob2char(feed)[0]
        print(sentence)
    print('=' * 80)